In [1]:
class HuffmanNode:
    def __init__(self, word_id, frequency):
        self.word_id = word_id
        self.frequency = frequency
        self.left_child = None
        self.right_child = None
        self.father = None
        self.Huffman_code = []
        self.path = []


class HuffmanTree:
    def __init__(self, wordid_frequency_dict):
        self.word_count = len(wordid_frequency_dict)
        self.wordid_code = dict()
        self.wordid_path = dict()
        self.root = None
        unmerge_node_list = [HuffmanNode(wordid, frequency) for wordid, frequency in wordid_frequency_dict.items()] # Unmerged node list
        self.huffman = [HuffmanNode(wordid, frequency) for wordid, frequency in wordid_frequency_dict.items()] # Store all leaf nodes and intermediate nodes
        # Build huffman tree
        self.build_tree(unmerge_node_list)
        # Generate huffman code
        self.generate_huffman_code_and_path()

    def merge_node(self, node1, node2):
        sum_frequency = node1.frequency + node2.frequency
        mid_node_id = len(self.huffman)
        father_node = HuffmanNode(mid_node_id, sum_frequency)
        if node1.frequency >= node2.frequency:
            father_node.left_child = node1
            father_node.right_child = node2
        else:
            father_node.left_child = node2
            father_node.right_child = node1
        self.huffman.append(father_node)
        return father_node

    def build_tree(self, node_list):
        while len(node_list) > 1:
            i1 = 0  # Node with least frequency
            i2 = 1  # Node with the second smallest frequency
            if node_list[i2].frequency < node_list[i1].frequency:
                [i1, i2] = [i2, i1]
            for i in range(2, len(node_list)):
                if node_list[i].frequency < node_list[i2].frequency:
                    i2 = i
                    if node_list[i2].frequency < node_list[i1].frequency:
                        [i1, i2] = [i2, i1]
            father_node = self.merge_node(node_list[i1], node_list[i2])  # Combine the least frequent two nodes
            if i1 < i2:
                node_list.pop(i2)
                node_list.pop(i1)
            elif i1 > i2:
                node_list.pop(i1)
                node_list.pop(i2)
            else:
                raise RuntimeError('i1 should not be equal to i2')
            node_list.insert(0, father_node)  # Insert new node
        self.root = node_list[0]

    def generate_huffman_code_and_path(self):
        stack = [self.root]
        while len(stack) > 0:
            node = stack.pop()
            while node.left_child or node.right_child:
                code = node.Huffman_code
                path = node.path
                node.left_child.Huffman_code = code + [1]
                node.right_child.Huffman_code = code + [0]
                node.left_child.path = path + [node.word_id]
                node.right_child.path = path + [node.word_id]
                stack.append(node.right_child)
                node = node.left_child
            word_id = node.word_id
            word_code = node.Huffman_code
            word_path = node.path
            self.huffman[word_id].Huffman_code = word_code
            self.huffman[word_id].path = word_path
            # Write the Huffman code and path calculated by the node into the value of the dictionary
            self.wordid_code[word_id] = word_code
            self.wordid_path[word_id] = word_path

    # Get the id of all positive nodes and the id of all negative nodes
    def get_all_pos_and_neg_path(self):
        positive = []  # Array of positive paths of all words
        negative = []  # Array of negative paths for all words
        for word_id in range(self.word_count):
            pos_id = []
            neg_id = []
            for i, code in enumerate(self.huffman[word_id].Huffman_code):
                if code == 1:
                    pos_id.append(self.huffman[word_id].path[i])
                else:
                    neg_id.append(self.huffman[word_id].path[i])
            positive.append(pos_id)
            negative.append(neg_id)
        return positive, negative


def test():
    word_frequency = {0: 4, 1: 6, 2: 3, 3: 2, 4: 2}
    print(word_frequency)
    tree = HuffmanTree(word_frequency)
    print(tree.wordid_code)
    print(tree.wordid_path)
    for i in range(len(word_frequency)):
        print(tree.huffman[i].path)
    print(tree.get_all_pos_and_neg_path())


if __name__ == '__main__':
    test()

{0: 4, 1: 6, 2: 3, 3: 2, 4: 2}
{1: [1, 1], 0: [1, 0], 3: [0, 1, 1], 4: [0, 1, 0], 2: [0, 0]}
{1: [8, 7], 0: [8, 7], 3: [8, 6, 5], 4: [8, 6, 5], 2: [8, 6]}
[8, 7]
[8, 7]
[8, 6]
[8, 6, 5]
[8, 6, 5]
([[8], [8, 7], [], [6, 5], [6]], [[7], [], [8, 6], [8], [8, 5]])


In [2]:
from collections import deque
import nltk
import re
from nltk.corpus import brown
from nltk.corpus import gutenberg
nltk.download('gutenberg')
nltk.download('stopwords')
nltk.download('brown')
nltk.download('punkt')


class InputData:
    def __init__(self, sentences):
        self.sentences = sentences
        self.normalize()
        self.counter = 0
        self.wordId_frequency_dict = dict()
        self.word_count = 0  # Number of words (repeated words only count as 1)
        self.word_count_sum = 0  # Total number of words (the number of repeated words also accumulates)
        self.sentence_count = 0  # Number of sentences
        self.id2word_dict = dict()
        self.word2id_dict = dict()
        self._init_dict()  # Initialize the dictionary
        self.huffman_tree = HuffmanTree(self.wordId_frequency_dict)  # Hoffman Tree
        self.huffman_pos_path, self.huffman_neg_path = self.huffman_tree.get_all_pos_and_neg_path()
        self.word_pairs_queue = deque()

        print('Word Count is:', self.word_count)
        print('Word Count Sum is', self.word_count_sum)
        print('Sentence Count is:', self.sentence_count)
        print('Tree Node is:', len(self.huffman_tree.huffman))

    def normalize(self):
      stop_words = nltk.corpus.stopwords.words('english')
      norm_sentences_word_list = []
      for word_list in self.sentences:
        sentence = " ".join(word for word in word_list)
        sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
        sentence = sentence.lower()
        sentence = re.sub(' +', ' ', sentence)
        sentence = sentence.strip()
        norm_word_list = sentence.split(' ')
        norm_word_list = [word for word in norm_word_list if word not in stop_words]
        if(len(norm_word_list) > 1):
          norm_sentences_word_list.append(norm_word_list)
        
      # print(self.sentences[5286])
      # print(norm_sentences_word_list[5274])  
      self.sentences = norm_sentences_word_list

    def _init_dict(self):
        word_freq = dict()
        for word_list in self.sentences:
            self.word_count_sum += len(word_list)
            self.sentence_count += 1
            for word in word_list:
                try:
                    word_freq[word] += 1
                except:
                    word_freq[word] = 1
        word_id = 0
        # Initialize word2id_dict, id2word_dict, wordId_frequency_dict dictionary
        for per_word, per_count in word_freq.items():
            self.id2word_dict[word_id] = per_word
            self.word2id_dict[per_word] = word_id
            self.wordId_frequency_dict[word_id] = per_count
            word_id += 1
        self.word_count = len(self.word2id_dict)

    def generate_context_word_pairs(self, window_size):
      self.counter += 1
      if not self.sentences[20*(self.counter-1):20*self.counter]:
        self.counter = 1
        self.word_pairs_queue.clear()
      sub_wids = [[self.word2id_dict[word] for word in word_list] for word_list in self.sentences[20*(self.counter-1):20*self.counter]]
      context_length = window_size*2
      for words in sub_wids:
          sentence_length = len(words)
          for index, word in enumerate(words):         
              start = index - window_size
              end = index + window_size + 1

              context_words = []
              for i in range(start, end):
                  if 0 <= i < sentence_length and i != index:
                    context_words.append(words[i])
                  elif i < 0 or i >= sentence_length:
                    context_words.append(self.word_count)

              self.word_pairs_queue.append((context_words,word))


    def get_batch_pairs(self, batch_size, window_size):

        while len(self.word_pairs_queue) < batch_size:
          self.generate_context_word_pairs(window_size)

        result_pairs = []  # Returns the positive sample pair of mini-batch size
        for _ in range(batch_size):
            result_pairs.append(self.word_pairs_queue.popleft())
        return result_pairs

    def get_pairs(self, pos_pairs):
        neg_word_pair = []
        pos_word_pair = []
        for pair in pos_pairs:
            pos_word_pair += zip([pair[0]] * len(self.huffman_pos_path[pair[1]]), self.huffman_pos_path[pair[1]])
            neg_word_pair += zip([pair[0]] * len(self.huffman_neg_path[pair[1]]), self.huffman_neg_path[pair[1]])
        return pos_word_pair, neg_word_pair


    def evaluate_pairs_count(self):
        return self.word_count_sum


# Test all methods
def test():
    sentences = brown.sents(categories=['news','reviews','humor','hobbies','fiction'])
    test_data = InputData(sentences)
    pos_pairs = test_data.get_batch_pairs(10, 2)
    print(sentences[0])
    print(sentences[1])
    print(pos_pairs)
    pos_word_pairs = []
    for pair in pos_pairs:
        pos_word_pairs.append(([test_data.id2word_dict[i] for i in pair[0] if i != test_data.word_count], test_data.id2word_dict[pair[1]]))
    print(pos_word_pairs)
    print('')
    print(test_data.huffman_pos_path[0])
    print(test_data.huffman_neg_path[0])
    pos, neg = test_data.get_pairs(pos_pairs)
    print(pos)
    print(neg)

    pos_word_pairs = []
    pos_pairs = test_data.get_batch_pairs(10, 2)
    for pair in pos_pairs:
        pos_word_pairs.append(([test_data.id2word_dict[i] for i in pair[0] if i != test_data.word_count], test_data.id2word_dict[pair[1]]))
    print(pos_pairs)
    print(pos_word_pairs)

    print(test_data.id2word_dict[4846])


if __name__ == '__main__':
    test()

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Word Count is: 24794
Word Count Sum is 147000
Sentence Count is: 15146
Tree Node is: 49587
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances


class CBOWModel(nn.Module):
    def __init__(self, emb_size, emb_dimension):
        super(CBOWModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(2*self.emb_size-1, self.emb_dimension, sparse=True)  # Define the embedded dictionary for the input word
        self.w_embeddings = nn.Embedding(2*self.emb_size-1, self.emb_dimension, sparse=True)  # Define the embedded dictionary for the output word
        self._init_embedding()  # initialization

    def _init_embedding(self):
        int_range = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-int_range, int_range)
        self.w_embeddings.weight.data.uniform_(-0, 0)

    def compute_context_matrix(self, u):
        pos_u_emb = self.u_embeddings(torch.LongTensor(u))
        pos_u_emb = torch.mean(pos_u_emb, 1, True)
        pos_u_emb = pos_u_emb.squeeze()

        return pos_u_emb

    def forward(self, pos_u, pos_w, neg_u, neg_w):
        pos_u_emb = self.compute_context_matrix(pos_u)
        pos_w_emb = self.w_embeddings(torch.LongTensor(pos_w))
        neg_u_emb = self.compute_context_matrix(neg_u)
        neg_w_emb = self.w_embeddings(torch.LongTensor(neg_w))

        score_1 = torch.mul(pos_u_emb, pos_w_emb).squeeze()
        score_2 = torch.sum(score_1, dim=1)
        score_3 = F.logsigmoid(score_2)
        neg_score_1 = torch.mul(neg_u_emb, neg_w_emb).squeeze()
        neg_score_2 = torch.sum(neg_score_1, dim=1)
        neg_score_3 = F.logsigmoid(-1 * neg_score_2)

        loss = torch.sum(score_3) + torch.sum(neg_score_3)
        return -1 * loss

    def distance_matrix(self, word_count):
        embedding = self.u_embeddings.weight.data.numpy()[:word_count]
        distance_matrix = euclidean_distances(embedding)
        return distance_matrix


def test():
    model = CBOWModel(100, 10)

    pos_u = [[9, 1, 2, 3],[0, 1, 2, 3]]
    pos_w = [50, 70]
    neg_u = [[9, 1, 2, 3],[0, 1, 2, 3]]
    neg_w = [30, 42]
    model.forward(pos_u, pos_w, neg_u, neg_w)
    distance_matrix = model.distance_matrix(5)
    print(distance_matrix.shape)


if __name__ == '__main__':
    test()

(5, 5)


In [189]:
import torch.optim as optim
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR

# hyper parameters
WINDOW_SIZE = 2 
BATCH_SIZE = 1000  # mini-batch
EMB_DIMENSION = 100  # embedding dimension
LR = 0.01  # Learning rate


class Word2Vec:
    def __init__(self,sentences):
        self.data = InputData(sentences)
        self.model = CBOWModel(self.data.word_count, EMB_DIMENSION)
        self.lr = LR
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.lr)
        lambda1 = lambda epoch: 0.99 ** epoch
        self.scheduler = LambdaLR(self.optimizer, lr_lambda=lambda1)

    def train(self):
        print("CBOW Training......")
        pairs_count = self.data.evaluate_pairs_count()
        print("pairs_count", pairs_count)
        batch_count = pairs_count / BATCH_SIZE
        print("batch_count", batch_count)
        for epoch in range(1,51):
            mean_loss = 0
            process_bar = tqdm(range(int(batch_count)))
            for i in process_bar:
                pos_pairs = self.data.get_batch_pairs(BATCH_SIZE, WINDOW_SIZE)
                pos_pairs, neg_pairs = self.data.get_pairs(pos_pairs)

                pos_u = [pair[0] for pair in pos_pairs]
                pos_v = [int(pair[1]) for pair in pos_pairs]
                neg_u = [pair[0] for pair in neg_pairs]
                neg_v = [int(pair[1]) for pair in neg_pairs]

                self.optimizer.zero_grad()
                loss = self.model.forward(pos_u,pos_v,neg_u,neg_v)
                loss.backward()
                self.optimizer.step()
                mean_loss += loss

            print("epoch:",epoch,"loss:",mean_loss/int(batch_count))
            self.scheduler.step()

    def get_distance_matrix(self):
        distance_matrix = self.model.distance_matrix(self.data.word_count)
        return distance_matrix

#def run():
    #sentences = brown.sents(categories=['news','reviews','humor','hobbies'])
    #w2v = Word2Vec(sentences)
    #w2v.train()
    
    #result = w2v.similar_words(['government','church','children','car','tax','food','election'])

    #print(result)            


#if __name__ == '__main__':
#    run()

In [190]:
sentences = brown.sents(categories=['news','reviews','government','hobbies','romance'])
w2v = Word2Vec(sentences)

Word Count is: 24616
Word Count Sum is 170964
Sentence Count is: 17106
Tree Node is: 49231


In [191]:
w2v.train()

CBOW Training......
pairs_count 170964
batch_count 170.964


100%|██████████| 170/170 [00:17<00:00,  9.90it/s]


epoch: 1 loss: tensor(8841.2080, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.65it/s]


epoch: 2 loss: tensor(8810.0869, grad_fn=<DivBackward0>)


...

epoch: 48 loss: tensor(6950.0889, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.68it/s]


epoch: 49 loss: tensor(6903.6265, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.96it/s]

epoch: 50 loss: tensor(6862.5098, grad_fn=<DivBackward0>)


In [234]:
w2v.train()

CBOW Training......
pairs_count 170964
batch_count 170.964


100%|██████████| 170/170 [00:16<00:00, 10.45it/s]


epoch: 1 loss: tensor(6822.8618, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.70it/s]


epoch: 2 loss: tensor(6776.9663, grad_fn=<DivBackward0>)


...

epoch: 48 loss: tensor(5171.8374, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.68it/s]


epoch: 49 loss: tensor(5152.9751, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:15<00:00, 10.85it/s]

epoch: 50 loss: tensor(5114.2725, grad_fn=<DivBackward0>)


In [249]:
w2v.train()

CBOW Training......
pairs_count 170964
batch_count 170.964


100%|██████████| 170/170 [00:16<00:00, 10.53it/s]


epoch: 1 loss: tensor(5102.5322, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:16<00:00, 10.56it/s]


epoch: 2 loss: tensor(5064.4951, grad_fn=<DivBackward0>)


...

epoch: 48 loss: tensor(4330.6133, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:16<00:00, 10.56it/s]


epoch: 49 loss: tensor(4327.4160, grad_fn=<DivBackward0>)


100%|██████████| 170/170 [00:16<00:00, 10.55it/s]

epoch: 50 loss: tensor(4310.2173, grad_fn=<DivBackward0>)


In [250]:
distance_matrix = w2v.get_distance_matrix()

In [251]:
print(w2v.data.word2id_dict)

{'fulton': 0, 'county': 1, 'grand': 2, 'jury': 3, 'said': 4, 'friday': 5, 'investigation': 6, 'atlantas': 7, 'recent': 8, 'primary': 9, 'election': 10, 'produced': 11, 'evidence': 12, 'irregularities': 13, 'took': 14, 'place': 15, 'termend': 16, 'presentments': 17, 'city': 18, 'executive': 19, 'committee': 20, 'overall': 21, 'charge': 22, 'deserves': 23, 'praise': 24, 'thanks': 25, 'atlanta': 26, 'manner': 27, 'conducted': 28, 'septemberoctober': 29, 'term': 30, 'charged': 31, 'superior': 32, 'court': 33, 'judge': 34, 'durwood': 35, 'pye': 36, 'investigate': 37, 'reports': 38, 'possible': 39, 'hardfought': 40, 'mayornominate': 41, 'ivan': 42, 'allen': 43, 'jr': 44, 'relative': 45, 'handful': 46, 'received': 47, 'considering': 48, 'widespread': 49, 'interest': 50, 'number': 51, 'voters': 52, 'size': 53, 'find': 54, 'many': 55, 'georgias': 56, 'registration': 57, 'laws': 58, 'outmoded': 59, 'inadequate': 60, 'often': 61, 'ambiguous': 62, 'recommended': 63, 'legislators': 64, 'act': 65, '

In [261]:
similar_words = {search_term: [w2v.data.id2word_dict[idx] for idx in distance_matrix[w2v.data.word2id_dict[search_term]].argsort()[1:10]] 
                   for search_term in ['tablespoon','election','sauce', 'democratic','player','game','children','mettwurst']}
similar_words

{'children': ['restudy',
  'nursery',
  'chassis',
  'equines',
  'scripts',
  'remedial',
  'kroger',
  'mingle',
  'uttermost'],
 'democratic': ['republican',
  'voted',
  'swell',
  'georgia',
  'bronx',
  'unworkable',
  'alamein',
  'defendants',
  'morocco'],
 'election': ['forwarded',
  'curiously',
  'boissoneault',
  'oregon',
  'lowliest',
  'independents',
  'conventions',
  'unanimous',
  'byrds'],
 'game': ['scored',
  'gaines',
  'popped',
  'baylors',
  'sometimesnecessary',
  'phonies',
  'sprained',
  'triplecrown',
  'upstanding'],
 'mettwurst': ['bratwurst',
  'photofloodlights',
  'lobster',
  'europeans',
  'jerusalem',
  'teaspoonful',
  'loneliness',
  'solitude',
  'toasted'],
 'player': ['burnings',
  'nagle',
  'impudent',
  'overloud',
  'outplayed',
  'winking',
  'incomparably',
  'dyer',
  'sermons'],
 'sauce': ['pineapple',
  'bread',
  'frankfurter',
  'minced',
  'tiers',
  'walnuts',
  'chicken',
  'franks',
  'needle'],
 'tablespoon': ['horseradish',
